In [ ]:
import py_entitymatching as em
import pandas as pd
import os
from utils.utils import *

from dataprep.eda import create_report
from IPython.display import display, HTML

pd.options.mode.chained_assignment = None  # default='warn''

<h3>0. Prelude.</h3>
This pipeline consists of the Magellan pipeline with intent of testing data. <br>
The pipeline can be adjusted for real data, and hybrid data. <br>
It takes into consideration GPT-2 data and Magellan data. <br>
With blocking and manual sampling as its own pipeline, this workflow focuses <br>
on getting the data to the matcher. Because of this, the blocking step is omitted. <br>

In [ ]:
# Set variables for this pipeline...
datasets_dir = r'C:\Users\aleks\Desktop\Master Thesis\Py_Magellan\DataSets\movies1'


# If we are testing synthethic data
testing_synth_data = True
if testing_synth_data:
    # True = GPT
    # False = CTGAN
    generator_type = False
    if generator_type:
        name_of_synth_data = "synth_data.txt"
        name_of_real_data = "real_data.txt"
    else:
        name_of_synth_data = "synth_data.csv"
        name_of_real_data = "real_data.csv"
else:
    # If we are testing DITTO data
    ditto_run = True
    train_proportion = 0.7
    if ditto_run:
        name_of_data = "data.txt"
    else:
        name_of_table_A = "imdb2.csv"
        name_of_table_B = "rotten_tomatoes2.csv"
        name_of_table_C = "sample_set.csv"



<h3>1. Load datasets.</h3>
First, load two datasets from our dataset path. <br>
We also include an ID generator for each line, as some of the datasets come without indices.

In [ ]:
if testing_synth_data:
    synth_data_path = datasets_dir + os.sep + name_of_synth_data
    real_data_path = datasets_dir + os.sep + name_of_real_data
    if generator_type:
        with open(synth_data_path, 'r', encoding='utf-8') as file:
            synth_data = file.read()

        with open(real_data_path, 'r', encoding='utf-8') as file:
            real_data = file.read()
        
        table_A_synth, table_B_synth, truth_synth = ditto_reformater(synth_data)
        # table1 = table_A.add_prefix("ltable_")
        # table2= table_B.add_prefix("rtable_")
        # table = pd.concat([table1, table2, table3], axis=1)
        table_A_real, table_B_real, truth_real = ditto_reformater(real_data)

        cutoff = len(table_A_synth.index)

        table_A = table_A_synth.append(table_A_real, ignore_index=True)
        table_B = table_B_synth.append(table_B_real, ignore_index=True)
        truth = truth_synth.append(truth_real, ignore_index=True)

    else:
        table_synth = pd.read_csv(synth_data_path)
        table_real = pd.read_csv(real_data_path)

        cutoff = len(table_synth.index)
        split_index = (len(table_synth.columns) - 1) / 2
        table = table_synth.append(table_real, ignore_index=True)

        truth = table.iloc[:,-1]
        table = table.iloc[:,:-1]
        table_A = table.iloc[:,:-split_index]
        table_B = table.iloc[:,-split_index:]

    A_meta = create_index_as_id_for_dataframe(table_A)
    B_meta = create_index_as_id_for_dataframe(table_B)
    em.set_key(A_meta, 'ID')
    em.set_key(B_meta, 'ID')

    table1 = A_meta.add_prefix("ltable_")
    table2 = B_meta.add_prefix("rtable_")
    table = pd.concat([table1, table2, truth], axis=1)
else:
    if ditto_run:
        data_path = datasets_dir + os.sep + name_of_data
        with open(data_path, 'r', encoding='utf-8') as file:
            data = file.read()
        
        table_A, table_B, truth = ditto_reformater(data)
        A_meta = create_index_as_id_for_dataframe(table_A)
        B_meta = create_index_as_id_for_dataframe(table_B)
        em.set_key(A_meta, 'ID')
        em.set_key(B_meta, 'ID')

        table1 = A_meta.add_prefix("ltable_")
        table2 = B_meta.add_prefix("rtable_")
        table = pd.concat([table1, table2, truth], axis=1)
    else:
        path_A = datasets_dir + os.sep + name_of_table_A
        path_B = datasets_dir + os.sep + name_of_table_B

        # Might need to remove the file reading from function and just read it here.
        create_index_as_id(path_A)
        create_index_as_id(path_B)
        A_meta = em.read_csv_metadata(path_A, key='ID')
        B_meta = em.read_csv_metadata(path_B,  key='ID')
        em.set_key(A_meta, 'ID')
        em.set_key(B_meta, 'ID')

if testing_synth_data:
    table_C_path = datasets_dir + os.sep + "Table_C.csv"
    table.to_csv(table_C_path)
else:
    if ditto_run:
        table_C_path = datasets_dir + os.sep + "Table_C.csv"
        table.to_csv(table_C_path)


<h3>2. Down-sizing.</h3>
Incase of the datasets being too large, we downsample the datasets before an official run.
This can be commented out for production stage.

In [ ]:
# A_meta, B_meta = em.down_sample(A_meta, B_meta, size=1000, y_param=1, show_progress=False)

In [ ]:
print(em._atypes1['Name'], em._atypes1['Genre'], em._atypes1['ReleaseDate'], em._atypes1['Creator'], em._atypes1['Director'])
print(em._atypes2['Name'], em._atypes2['Genre'], em._atypes2['ReleaseDate'], em._atypes2['Creator'], em._atypes2['Director'])
em._atypes1['Genre'] = 'str_bt_1w_5w'
em._atypes2['Genre'] = 'str_bt_1w_5w'
atypes1 = em.get_attr_types(A_meta)
atypes2 = em.get_attr_types(B_meta)

block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
block_c = em.get_attr_corres(A_meta, B_meta)

# Then we use the command to see which columns are comparable.
print(block_c)
feature_table  = em.get_features(A_meta, B_meta, atypes1, atypes2, block_c, block_t, block_s )


<h4>4. Sample data to be labelled</h4>
For our next step, we need to sample some data and label it accordingly for our matcher. <br>
Sample set amount can vary, but these are to be our ground-truth forward. <br> <br> 
As of 2022, Magellan does not support proper GUI for labelling, so we save the sample set to a filepath and label it manually.  <br>
Then, we load the dataset from its origin path.

In [ ]:
path_C = datasets_dir + os.sep + 'Table_C.csv'
A_meta.fillna(0)
B_meta.fillna(0)
C_meta = em.read_csv_metadata(path_C, key='_id',
                                    fk_ltable='ltable_ID', fk_rtable='rtable_ID',
                                    ltable=A_meta, rtable=B_meta)


print("Number of matches: " + str(sum(C_meta['Truth'] > 0)))
print("Number of non-matches: " + str(sum(C_meta['Truth'] < 1)))
# create_report(C_meta).show()

<h4>5. Selecting and training a matcher on labelled data. </h4>
Furthermore, we use the labelled data to train our matcher. <br> <br>

1. We start off by splitting our labelled data into a training set, and a test set. <br>
2. Secondly, we generate the set of features. For now, we will use py_entitymatchings native function to automatically generate these features. <br>
3. Then, we convert the labeled data to feature vectors using the feature table. <br> 
4. We also handle missing data / NaN values...

In [ ]:
if testing_synth_data:
    dataset_train = C_meta.iloc[:cutoff,:]
    dataset_test = C_meta.iloc[cutoff:,:]
    # Update catalog
    em.catalog.catalog_manager.init_properties(dataset_train)
    em.catalog.catalog_manager.copy_properties(C_meta, dataset_train)

    em.catalog.catalog_manager.init_properties(dataset_test)
    em.catalog.catalog_manager.copy_properties(C_meta, dataset_test)
else:
    dataset_split = em.split_train_test(C_meta, train_proportion=train_proportion)
    dataset_train = dataset_split['train']
    dataset_test = dataset_split['test']



# dataset_train = C_meta
# dataset_test = C_meta2

print(" ========= Training =========")
print("Number of matches: " + str(sum(dataset_train['Truth'] > 0)))
print("Number of non-matches: " + str(sum(dataset_train['Truth'] < 1)))
print(" ========= Validation =========")
print("Number of matches: " + str(sum(dataset_test['Truth'] > 0)))
print("Number of non-matches: " + str(sum(dataset_test['Truth'] < 1)))

feature_table_train = em.get_features_for_matching(A_meta, B_meta, validate_inferred_attr_types=False)

# Convert the labeled data to feature vectors using the feature table
feature_vector_table_train = em.extract_feature_vecs(dataset_train, 
                            feature_table=feature_table_train, 
                            attrs_after='Truth',
                            show_progress=False)

# Handle missing data. Inplace is required to keep correct object type, and maintaining "Meta-Data".
feature_vector_table_train.fillna(value=0, inplace=True)

<h4>5.1. Selecting the best matcher.</h4>

In [ ]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree')
svm = em.SVMMatcher(name='SVM')
rf = em.RFMatcher(name='RF')
lg = em.LogRegMatcher(name='LogReg')
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [ ]:
# Select the best ML matcher using Cross-Validation
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=feature_vector_table_train, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'],
        k=5,
        target_attr='Truth', metric_to_select_matcher='precision') # Can be switched out for 'recall'?
result['cv_stats']

We choose Random-Forest for testing purposes.

<h4>5.2. Training the selected matcher.</h4>

In [ ]:
# Instantiate the RF Matcher
rf = em.RFMatcher()

# Get the attributes to be projected while training
attrs_to_be_excluded = []
attrs_to_be_excluded.extend(['_id', 'ltable_ID', 'rtable_ID'])

# Train using feature vectors from the labeled data.
rf.fit(table=feature_vector_table_train, exclude_attrs=attrs_to_be_excluded, target_attr='Truth')

<h4>6. Predict the matches on the evaluation set using the trained matcher.</h4>


In [ ]:
feature_vector_table_test = em.extract_feature_vecs(dataset_test, feature_table=feature_table,
                             attrs_after='Truth',
                             show_progress=False)

# Handle NaN values.
feature_vector_table_test.fillna(value=0, inplace=True)

In [ ]:
# Get the attributes to be excluded while predicting
attrs_to_be_excluded = []
attrs_to_be_excluded.extend(['_id', 'ltable_ID', 'rtable_ID', 'Truth'])

# Predict the matches
predictions = rf.predict(table=feature_vector_table_test, exclude_attrs=attrs_to_be_excluded,                          
              append=True, target_attr='predicted', inplace=False)

In [ ]:

# Project the attributes
predictions.head(10)

# Show only rows which are predicted as "matched".
df_only_matched = predictions.loc[predictions['predicted'] == 1]
df_only_matched.head(100)

# Evaluate prediction results.
eval_result = em.eval_matches(predictions, 'Truth', 'predicted')
em.print_eval_summary(eval_result)